In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.io import fits
from astropy.cosmology import WMAP9 as cosmo
import astropy.units as u
import astropy.constants as const
from scipy.optimize import fsolve
from scipy.interpolate import CubicSpline

### look for duals in some sep bin

In [ ]:
df2 = pd.read_pickle("/home/insepien/research-data/pop-result/separation.pkl")
sep_kpc = df2['sep_kpc']

def find_agn_in_bin(bin_num):
    bins = np.logspace(-1,1.5,10)
    bin0_inds = np.where((bins[bin_num] <= sep_kpc) & (sep_kpc <= bins[bin_num+1]))
    bin0_agns = df2.loc[[i if i<14 else i+1 for i in bin0_inds[0]]]
    return bin0_agns

#find_agn_in_bin(2)
df2

### check J1010 calculations

In [ ]:
k = [j for j in j1010.keys() if j[:len('OIII_5007')] =='OIII_5007']
[print(key, j1010[key].values[0]) for key in k];

In [ ]:
j1010['OIII_5007_LUM_DERRED']

In [ ]:
from astropy.cosmology import Planck13
## J1010 CHECKS
data = fits.getdata('/home/insepien/research-data/alpaka/ALPAKA_v1_withDes.fits')
# get j1010 row
alpaka = Table(data).to_pandas()
j1010 = alpaka[alpaka['Desig']=="J1010+1413"]
lums = [i for i in list(j1010.keys()) if 'LUM' in i]

# OIII luminosity and bolo from OIII
ja_oiii = 1.2e44 # different from alpaka value??
i1 = ja_oiii*800/(1+1.3)
i2 = i1*1.3
print(f"alpaka OIII lum extinction corrected: {j1010['OIII_5007_LUM_DERRED'].values[0]:.1e}")
print(f"paper OIII lum: {ja_oiii:.1e}")
print(f"by paper OIII lum, my core 1 bolometric: {i1:.1e}, my core 2 bolometric: {i2:.1e}")

# IR  luminosity 
f160 = 3.23e-12*u.erg/u.s/(u.cm)**2 # from paper
ja_ir = 1.1e11
da = Planck13.luminosity_distance(j1010['Z'].values[0])
print(f"paper IR lum: {ja_ir:.1e}")
print(f"my IR Lum: {(f160*4*np.pi*da**2).to(u.L_sun):.1e}")

In [ ]:
# checking J1010 bolometric luminosity using WISE mIR
wmags_ = np.array([13.056, 11.208, 6.784, 3.816])

zeromagflux = np.array([309.540, 171.787, 31.674, 8.363])*u.Jy
fluxdens = zeromagflux*10**(-wmags_/2.5)
## get flux at obs wavelength
obs_flux = fluxdens[-1]
## change to luminosity
obs_hz = (const.c/(22*u.micron)).to(u.Hz)
ja_ir = 6e46
mylum = (obs_flux*obs_hz*4*np.pi*Planck13.luminosity_distance(0.2)**2).to(u.erg/u.s).value
print(f"paper IR lum?: {ja_ir:.1e}")
print(f"IR Lum: {mylum*1.12:.1e}")

In [ ]:
with open("/home/insepien/research-data/pop-result/bc.txt","r") as f:
    d = f.read().splitlines()
hopkins = pd.DataFrame([d[1:][i].split(' ') for i in range(len(d[1:]))],columns=d[0].split(' '))
Lbol = np.array(list(hopkins['Lbols'].values), dtype=float)
LIR = np.array(list(hopkins['LIRs'].values), dtype=float)

# interpolate wise IR lum
from scipy.interpolate import CubicSpline
spl = CubicSpline(LIR, Lbol)
10**(spl(np.log10(mylum)))

### calculate bolometric luminosity from WISE mIR and Hopkins+20 corrections to 15 nm

In [ ]:
def get_wise_mags(wise_):
    """get wise mags and mag errors from data frame of ipac search results"""
    ## get wise mags and errors
    w1mag = wise_['w1mpro']
    w2mag = wise_['w2mpro']
    w3mag = wise_['w3mpro']
    w4mag = wise_['w4mpro']
    wmags_ = np.array([w1mag, w2mag, w3mag, w4mag])
    wmags_err_ = np.array([wise_['w1sigmpro'], wise_['w2sigmpro'], wise_['w3sigmpro'], wise_['w4sigmpro']])
    return wmags_, wmags_err_


def wise_lum_from_mag(wmags_, wmags_err_, rf_wavelength_, redshift_):
    """turn wise mags into wise flux at restframe 15 microns"""
    ## get observed wavelength corresponding to global rest-frame wavelength
    obs_wavelength = rf_wavelength_#*(1+redshift_)
    ## change mags to fluxes -- http://wise2.ipac.caltech.edu/docs/release/allsky/expsup/sec4_4h.html#example
    zeromagflux = np.array([309.540, 171.787, 31.674, 8.363])*u.Jy
    fluxdens = zeromagflux*10**(-wmags_/2.5) # in Jy
    fluxdens_err = zeromagflux*10**(-wmags_err_/2.5)
    ## interpolate - use straight line
    wise_wavelengths = np.array([3.4, 4.6, 12., 22.]) # 1e-6 m
    wiseflux = np.polyfit(wise_wavelengths, fluxdens.value,1, w=1./fluxdens_err)
    ## get flux at obs wavelength, i.e. just a straight line here
    obs_flux = (wiseflux[0]*obs_wavelength+wiseflux[1])*u.Jy
    ## change to luminosity
    obs_hz = (const.c/(obs_wavelength*u.micron)).to(u.Hz)
    lum = (obs_flux*obs_hz*4*np.pi*
           cosmo.luminosity_distance(redshift_)**2).to(u.erg/u.s)
    return lum


def correct_ir():
    """correct IR luminosity at 15 microns rest frame based on Hopkins+20"""
    # load hopkins bolometric correction
    with open("/home/insepien/research-data/pop-result/bc.txt","r") as f:
        d = f.read().splitlines()
    hopkins = pd.DataFrame([d[1:][i].split(' ') for i in range(len(d[1:]))],columns=d[0].split(' '))
    Lbol = np.array(list(hopkins['Lbols'].values), dtype=float)
    LIR = np.array(list(hopkins['LIRs'].values), dtype=float)
    spl = CubicSpline(LIR, Lbol)
    return spl

Check with J1010 wise lum

In [ ]:
paper_ir_bol = 6e46
print(f"paper bolometric luminosity from WISE IR: {paper_ir_bol:.2e}")
j10_ipac = pd.read_csv("/home/insepien/research-data/alpaka/j1010_ipac.csv")
wmag10, wmagerr10  = get_wise_mags(j10_ipac)
j10_w_lum = wise_lum_from_mag(wmag10[:,0], wmagerr10[:,0], 22, 0.198).value
j10_ir_bol = j10_w_lum*10**1.2#10**(spl(np.log10(j10_w_lum)))
print(f"J1010 AGN lum from WISE IR at 22 microns: {j10_ir_bol:.2e}")

In [ ]:
# this file is from query of sources on WISE, instructions for query in kris candidate selection notebook
# wise search has 39 objects fitted in magellan sample
wise = pd.read_pickle("/home/insepien/research-data/alpaka/wise_39fits.pkl")
# read alpaka for optical comparison and redshift
# alpaka has 41 rows, since J0926+0724 (1 is non-agn) and J1222-0007 (dual) are duplicated
alpaka = pd.read_pickle("/home/insepien/research-data/alpaka/alpaka_39fits.pkl")
alpaka.reset_index(inplace=True)
## get wise mags and errors
wmags, wmags_err_nan = get_wise_mags(wise)
# replace nan values in mag error with median
wmags_err = np.nan_to_num(wmags_err_nan,np.median(wmags_err_nan))
# calculate 15 microns lum
wise_lums = np.zeros((len(wise)))
for i in range(0, len(wise)):
    on = wise.loc[i,"designation"]
    z = np.mean(alpaka[alpaka['Desig']==on]['Z'])
    wise_lums[i] = wise_lum_from_mag(wmags[:,i], wmags_err[:,i], 22, alpaka.loc[i,"Z"]).value

# bolometric correction from Hopkins
# spl = correct_ir()
# # interpolate wise IR lum
# irbol = 10**(spl(np.log10(wise_lums)))

irbol = wise_lums*10**1.12

In [ ]:
# compare OIII and IR bolo lum
df_all = pd.read_pickle("/home/insepien/research-data/pop-result/all_results_updated.pkl")
#oiiibol = alpaka['OIII_5007_LUM_DERRED']*800
oiilbol = df_all['OIII_BOL']

In [ ]:
def read_mul171():
    # read mullaney and add designations
    mul171_ = pd.read_pickle("/home/insepien/research-data/alpaka/mull171.pkl")
    designations = []
    for i in range(0, len(mul171_)):
        pos = SkyCoord(ra=mul171_['RA'][i]*u.deg, dec=mul171_['DEC'][i]*u.deg)
        posstring = pos.to_string('hmsdms').split(' ')
        des_ra = posstring[0][0:2]+posstring[0][3:5]
        des_dec = posstring[1][0:3]+posstring[1][4:6]
        des = 'J'+des_ra+des_dec
        designations.append(des)
    mul171_['DESIG'] = designations
    return mul171_

mul171 = read_mul171()
mul171[mul171['DESIG']=='J1550+4005'][['RA',"DEC"]]

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(13,4))
ax[0].hist(pd.DataFrame(wmags).loc[3], label='magellan sample', color='orange') 
ax[0].axvline(wmag10[-1], label='J1010',c='steelblue')
ax[0].set_title("WISE magnitude of Magellan sample")
ax[0].legend()
ax[0].set_xlabel("Vega Mag at 22 microns")

ax[1].hist(np.log10(oiiibol),label="OIII*800",color='steelblue')
ax[1].axvline(np.log10(1.2e43*800),label='j1010 OIII*800',c="steelblue")

ax[1].hist(np.log10(oiiibol/800*100),label="OIII*100",color='green')
ax[1].axvline(np.log10(1.2e43*100),label='j1010 OIII*100',c="green")

ax[1].hist(np.log10(irbol),label='wise IR corrected by 12%', color='orange')
ax[1].axvline(np.log10(paper_ir_bol),label='j1010 IR',c="orange")

ax[1].set_xlabel("Log(L_bol)")
ax[1].set_ylabel("# quasars")
ax[1].set_title("bolometric luminosity")
ax[1].legend()
;


Hi Kris, I have fixed the code, and my bolometric luminosity of J1010 at 22 microns matches Andy's number (6e46). However, the Magellan sample is dimmer than J1010 (left panel), so it is consistent that the IR bolometric luminosity is lower than e46 (right panel). OIII and IR would be consistent if we correct OIII by 100 rather than 800. Also I am using OIII value straight from Mullaney. In J1010 paper, Andy use a number that is 10x higher. I remember they said this number is from some other catalog, but that's strange because they wrote the luminosity exactly like Mullaney (L_OIII_dered) and quoted Mullaney right before that. If you think this result is ok, I can draft an email to Andy to ask where exactly the OIII value he used is from. 